## 此版本现在多了confusion matrix, 取消l2, 进行finetune

In [18]:
from torchvision import transforms, utils
from torchvision import datasets
import torch
import matplotlib.pyplot as plt 
from torch.utils import data
import random
import numpy as np
import torch.nn as nn
import torch.optim as optim
import time
from torch.utils.data.sampler import SubsetRandomSampler
import copy
from sklearn.metrics import confusion_matrix

## load data

In [19]:
# def splitData(datadir,val_size = 0.2):
    
#     train_trainsforms = transforms.Compose([transforms.RandomResizedCrop(299),
#                 transforms.RandomHorizontalFlip(),
#                 transforms.RandomRotation(30),
#                 transforms.ToTensor(),
#                 #transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))])
#                 transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))])
#     val_trainsforms = transforms.Compose([transforms.RandomResizedCrop(299),
#                 transforms.ToTensor(),
#                 transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))
# ])

#     train_data = datasets.ImageFolder(datadir,transform=train_trainsforms)
#     val_data = datasets.ImageFolder(datadir,transform=val_trainsforms)

#     num = len(train_data)                              
#     idx = list(range(num))                         
#     split = int(np.floor(val_size * num))         
#     np.random.shuffle(idx)                              

#     val_idx, train_idx = idx[:split], idx[split:]
#     train_sampler = SubsetRandomSampler(train_idx)            
#     val_sampler  = SubsetRandomSampler(val_idx)
#     train_loader = data.DataLoader(train_data,sampler=train_sampler,batch_size=64)
#     val_loader = data.DataLoader(val_data,sampler=val_sampler,batch_size=64)
    
#     return train_loader,val_loader


In [20]:
def make_weights_for_balanced_classes(images, nclasses):                        
    count = [0] * nclasses                                                      
    for item in images:                                                         
        count[item[1]] += 1                                                     
    weight_per_class = [0.] * nclasses                                      
    N = float(sum(count))                                                   
    for i in range(nclasses):                                                   
        weight_per_class[i] = 1/float(count[i])                                 
    weight = [0] * len(images)                                              
    for idx, val in enumerate(images):                                          
        weight[idx] = weight_per_class[val[1]]  
    print(N)
    print(weight_per_class)
    return weight

In [21]:
#train_data_dir = 'OCT2017/test/'
#train_loader, val_loader = splitData(train_data_dir,val_size = 0.2)
batch_size = 128
#train_size = 1600
train_data = datasets.ImageFolder(root='OCT2017/train/', transform=transforms.Compose([
    transforms.CenterCrop(299),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(30),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))
]))

val_data = datasets.ImageFolder(root='OCT2017/val/', transform=transforms.Compose([
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))
]))

test_data = datasets.ImageFolder(root='OCT2017/test/', transform=transforms.Compose([
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))
]))

weights = make_weights_for_balanced_classes(train_data.imgs, len(train_data.classes))
weights = torch.DoubleTensor(weights)
weight_sampler = torch.utils.data.sampler.WeightedRandomSampler(weights, len(weights))
#weight_sampler = torch.utils.data.sampler.WeightedRandomSampler(weights, train_size)

train_loader = data.DataLoader(train_data,
                               batch_size=batch_size,
                               sampler = weight_sampler
                                            )
       

val_loader = data.DataLoader(val_data,
                             batch_size=batch_size,
                             shuffle=True,
                                        )


test_loader = data.DataLoader(test_data,
                                            batch_size=batch_size,
                                            shuffle=True,
                                            )
       


82484.0
[2.7059937762143145e-05, 9.010632546404758e-05, 0.00011953143676786995, 3.836562440053712e-05]


In [23]:
print(len(train_loader.dataset))
82484.0
[2.232011906372615, 7.4323301495765, 9.859431030360986, 3.164550163053904]


82484


[2.232011906372615, 7.4323301495765, 9.859431030360986, 3.164550163053904]

## load pre-trained model

In [24]:
import torchvision.models as models
inception = models.inception_v3(pretrained=True)


In [25]:
print(inception)

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, t

## parameter setting

In [26]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False
#     else:
#         for parma in model.parameters():
#             parma.requires_grad = False
#         for parma in model.Mixed_7c.parameters():
#             parma.requires_grad=True
#         for parma in model.Mixed_7b.parameters():
#             parma.requires_grad=True
#         for parma in model.Mixed_7a.parameters():
#             parma.requires_grad=True
#     else: 
#         for param in model.parameters():
#             param.requires_grad = False
#         ct = []
#         for name, child in model.named_children():
#             if "Conv2d_4a_3x3" in ct:
#                 for params in child.parameters():
#                     params.requires_grad = True
#             ct.append(name)       
    else:     
        for param in model.parameters():
            param.requires_grad = True

In [27]:
feature_extract = False
para = set_parameter_requires_grad(inception, feature_extract)

num_ftrs = inception.fc.in_features
#inception.fc=nn.Linear(num_ftrs, out_features=4, bias=True)
fc1 = nn.Linear(num_ftrs, out_features=1000, bias=True)
inception.fc = nn.Sequential(nn.Linear(num_ftrs, out_features=1000, bias=True),
                              nn.Linear(1000, out_features=4, bias=True)
                                 )
num_ftrs= inception.AuxLogits.fc.in_features
inception.AuxLogits.fc=nn.Linear(num_ftrs, out_features=4, bias=True)




## optimizer setting

In [28]:
params_to_update = inception.parameters()
weight_p, bias_p = [],[]
print("Params to learn:")
if feature_extract:
    print("tune fc layer")
    params_to_update = []
    for name,param in inception.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            
else:
    print("fine tune:")
    for name,param in inception.named_parameters():
        if param.requires_grad == True:
            print("\t",name)
           
        

# Observe that all parameters are being optimized
#optimizer_inception = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

optimizer_inception = optim.Adam(list(filter(lambda p: p.requires_grad, inception.parameters())), lr=5e-4, betas=(0.9, 0.99))
#optimizer_inception = optim.Adam(params_to_update, lr=0.001, betas=(0.9, 0.99))
#optimizer_inception = optim.Adam([{'params': weight_p, 'weight_decay':1e-3},
#                      {'params': bias_p, 'weight_decay':0}], lr=0.001, betas=(0.9, 0.99))

Params to learn:
fine tune:
	 Conv2d_1a_3x3.conv.weight
	 Conv2d_1a_3x3.bn.weight
	 Conv2d_1a_3x3.bn.bias
	 Conv2d_2a_3x3.conv.weight
	 Conv2d_2a_3x3.bn.weight
	 Conv2d_2a_3x3.bn.bias
	 Conv2d_2b_3x3.conv.weight
	 Conv2d_2b_3x3.bn.weight
	 Conv2d_2b_3x3.bn.bias
	 Conv2d_3b_1x1.conv.weight
	 Conv2d_3b_1x1.bn.weight
	 Conv2d_3b_1x1.bn.bias
	 Conv2d_4a_3x3.conv.weight
	 Conv2d_4a_3x3.bn.weight
	 Conv2d_4a_3x3.bn.bias
	 Mixed_5b.branch1x1.conv.weight
	 Mixed_5b.branch1x1.bn.weight
	 Mixed_5b.branch1x1.bn.bias
	 Mixed_5b.branch5x5_1.conv.weight
	 Mixed_5b.branch5x5_1.bn.weight
	 Mixed_5b.branch5x5_1.bn.bias
	 Mixed_5b.branch5x5_2.conv.weight
	 Mixed_5b.branch5x5_2.bn.weight
	 Mixed_5b.branch5x5_2.bn.bias
	 Mixed_5b.branch3x3dbl_1.conv.weight
	 Mixed_5b.branch3x3dbl_1.bn.weight
	 Mixed_5b.branch3x3dbl_1.bn.bias
	 Mixed_5b.branch3x3dbl_2.conv.weight
	 Mixed_5b.branch3x3dbl_2.bn.weight
	 Mixed_5b.branch3x3dbl_2.bn.bias
	 Mixed_5b.branch3x3dbl_3.conv.weight
	 Mixed_5b.branch3x3dbl_3.bn.weight
	

In [29]:
print(len(list(filter(lambda p: p.requires_grad, inception.parameters()))))

294


In [30]:
def exp_lr_scheduler(optimizer, epoch, init_lr=5e-4, lr_decay_epoch=10):
    """Decay learning rate by a factor of 0.1 every lr_decay_epoch epochs."""
    lr = init_lr * (0.3**(epoch // lr_decay_epoch))
    #lr = init_lr
    if epoch % lr_decay_epoch == 0:
        #lr = lr * 0.1
        print('LR is set to {}'.format(lr))
 
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
 
    return optimizer


## train model

In [31]:
use_cuda = 1
if use_cuda:
    inception = inception.cuda()

In [32]:
def train(model, trainloaders, valloaders, criterion, optimizer, lr_scheduler, num_epoch = 3):
    since =time.time()
    val_acc_history = []
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    train_loss, val_loss = [], []
    
    for epoch in range(num_epoch):
        
        optimizer = lr_scheduler(optimizer, epoch)
        steps = 0
        print('Epoch {}/{}'.format(epoch+1, num_epoch))
        print('-'*10)
                
        running_loss = 0.0
        running_corrects = 0 
        for inputs, labels in trainloaders:
            #print(labels.data)
            model.train()
            steps += 1
            #print(steps)
            
            inputs = inputs.cuda()
            labels = labels.cuda()

            optimizer.zero_grad()
            
            outputs, aux_outputs = model(inputs)
            loss_outputs = criterion(outputs, labels)
            loss_aux_outputs = criterion(aux_outputs, labels)
            loss = loss_outputs + 0.1*loss_aux_outputs

            _,preds = torch.max(outputs,1)

            loss.backward()
            optimizer.step()
    
            running_loss += loss.item()
            running_corrects += torch.sum(preds == labels.data)
            
        val_running_loss = 0.0
        val_running_corrects = 0
        model.eval()
        step = 0
        with torch.no_grad():
            for inputs, labels in valloaders:
                step += 1
                #print(step)
                inputs = inputs.cuda()
                labels = labels.cuda()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_running_loss += loss.item()
                _,preds = torch.max(outputs,1)
                #print(preds)
                val_running_corrects += torch.sum(preds == labels.data)

        train_loss.append(running_loss / len(trainloaders))
        val_loss.append(val_running_loss / len(valloaders))
        
        train_acc = running_corrects.double() / len(trainloaders.dataset)
        val_acc = val_running_corrects.double() / len(valloaders.dataset)
        
        print(f"Train loss: {running_loss / len(trainloaders):.3f}.."
              f"Train accuracy:{train_acc:.3f}.."
              f"Val loss: {val_running_loss/len(valloaders):.3f}.."
              f"Val accuracy: {val_acc:.3f}.."
             )

                
        # deep copy the model
        if val_acc > best_acc:
            best_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            val_acc_history.append(val_acc)

   # print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    conf_matrix = confusion_matrix(preds.data.cpu().numpy(), labels.data.cpu().numpy())
    print(conf_matrix)
    # load best model weights
    model.load_state_dict(best_model_wts)
    
    #return model, val_acc_history  
    return model, val_acc_history, train_loss, val_loss, conf_matrix
    

In [33]:
criterion = nn.CrossEntropyLoss()
optimizer = optimizer_inception
_, hist,train_loss,val_loss,cm = train(inception, train_loader, val_loader, criterion, optimizer, exp_lr_scheduler, num_epoch = 30)

plt.plot(train_loss, label='Training loss')
plt.plot(val_loss, label='Validation loss')
plt.legend(frameon=False)
plt.savefig("model_3.png")
plt.show()





LR is set to 0.0005
Epoch 1/30
----------
Train loss: 14.334..Train accuracy:0.916..Val loss: 0.167..Val accuracy: 0.945..
Epoch 2/30
----------


KeyboardInterrupt: 

In [ ]:
print(cm)

In [ ]:
def inference(model, testloaders, criterion):
    test_running_corrects = 0
    test_running_loss = 0
    with torch.no_grad():
        for inputs, labels in testloaders:
            inputs = inputs.cuda()
            labels = labels.cuda()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_running_loss += loss.item()
            _,preds = torch.max(outputs,1)
        
            test_running_corrects += torch.sum(preds == labels.data)

    test_acc = test_running_corrects.double() / len(testloaders.dataset)
    conf_matrix = confusion_matrix(preds.data.cpu().numpy(), labels.data.cpu().numpy())
    print(conf_matrix)
    print(f"Loss of the network on the 1000 test images: {test_running_loss/len(testloaders):.3f}.."
          f"Accuracy of the network on the 1000 test images: {test_acc:.3f}.."
         )

In [ ]:
inference(inception, test_loader, criterion)

In [ ]:
inference(inception, test_loader, criterion)